# Creating Eigenfaces & Recognizing Faces from training set using cv2 functions

In [1]:
# Import necessary packages.
from __future__ import print_function
import os
import sys
import cv2
import numpy as np
from skimage.transform import resize
import imageio
from PIL import Image

In [2]:
# print ("Loading Trained model")
# # Load the trained eigenfaces model
# model = cv2.face.EigenFaceRecognizer_create()
# model.read('eigenfaces_model.xml')

#TUNEABLE PARAMS

# newshape = (106, 128) #(160,121)
newshape = (195, 231) #(160,121)
newshape = (50, 52)

numProjectionsSkipped = 0
data = np.genfromtxt('MeanAndFaces.csv', delimiter=',') #make eigencsv for eigen
mean = data[0]
eigenmatrix = data[1+numProjectionsSkipped:]
dataACR = np.genfromtxt('ACR.csv', delimiter=',')[:,numProjectionsSkipped:] # make eigenACR for eigen
roiwidth = newshape[0]
roiheight = newshape[1]

#FOR 50x52
roiwidth = newshape[0] *3
roiheight = newshape[1]*3



In [3]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
numPicsPerPerson = 3
labels = np.zeros(numPicsPerPerson*3)
for i in range(3):
    labels[i*numPicsPerPerson:i*numPicsPerPerson+numPicsPerPerson] = i

# clf = SVC(kernel='linear', gamma='auto')
# clf.fit(dataACR, labels)
# labels = np.array([0,0,0,0,0,1,1,1,1,1,2,2,2,2,2])
# print(labels.shape)
# print(data.shape)
#TEMPORARY SINCE NO AAHAN PRESENT
# labels = labels[3:]
# data = data[3:]
# print(labels.shape)
# print(data.shape)
# Create a KNN classifier with k=3
# knn = KNeighborsClassifier(n_neighbors=1)

# Fit the classifier to the data
# knn.fit(data, labels)


In [4]:
# Load video, 0 for webcam, str for path to video
video = cv2.VideoCapture(0)
# video = cv2.VideoCapture('test_clip.mp4')

# Exit if video not opened.
if not video.isOpened():
    print('Could not open video!')
    sys.exit()

# Tracker Variables
tracker = None
roi = (0, 0, 0, 0)
# -1 for not tracking, 0 for init tracking, 1 for update tracking
tracking_flag = -1

counter = 0
# Loop simulate Camera Preview Callback
while True:
    
    # Capture user Key Press to simulate App Control
    key = cv2.waitKey(1) & 0xff
    # User Press Enter
    if key == 13:
        # Not tracking
        if tracking_flag == -1:
            # Pause and let user select ROI
#             roi = cv2.selectROI(frame, False) # Manual Setting of ROI in seperate window
#             print("ROI: ", roi)
            print("Frame Shape: ", frame.shape) # Frame Shape:  (480, 640, 3)
            
            # Init tracking
            tracking_flag = 0
        # Is tracking
        if tracking_flag == 1:
            # Reset ROI
            roi = (0, 0, 0, 0)
            # Clear Tracker
            tracker.clear()
            # Stop tracking
            tracking_flag = -1
    # User Press ESC
    elif key == 27:
        break
    
    # Start timer
    start = cv2.getTickCount()

    # Read Next frame.
    read_success, frame = video.read()
    if not read_success:
        print('Cannot read video file!')
        sys.exit()

    if tracking_flag == -1:

        # Display Text
        cv2.putText(frame, "Press ENTER to select ROI!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
        
        # Get frame dimensions
        height, width, _ = frame.shape

        w = roiwidth # The width of the selected ROI.
        h = roiheight # The height of the selected ROI.            
        x = int(width/2 - w/2) # The x-coordinate of the top-left corner of the selected region of interest (ROI).
        y = int(height/2 - h/2) # The y-coordinate of the top-left corner of the selected ROI.

        # (218 # of rows, 178 # of columns)

        roi = (x, y, w, h)

        # Draw ROI Rectangle
        p1 = (int(roi[0]), int(roi[1]))
        p2 = (int(roi[0] + roi[2]), int(roi[1] + roi[3]))
        cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)


    elif tracking_flag == 0:

        # Initialize KCF Tracker and Start Tracking
        # 1. Create a KCF Tracker
        # 2. Initialize KCF Tracker with grayscale image and ROI
        # 3. Modify tracking flag to start tracking
        # Your code starts here
        #cv2.getTickFrequency()
        # print("ROI: ", roi) # ROI:  (240, 320, 178, 218)
        tracker = cv2.TrackerKCF_create()
        status = tracker.init(frame,roi)
        
        tracking_flag = 1


    else:
        counter+=1
        # Update tracking result is succeed
        # If failed, print text "Tracking failure occurred!" at top left corner of the frame
        # Calculate and display "FPS@fps_value" at top right corner of the frame
        # Your code starts here

        read_success, roi = tracker.update(frame)
        
        # Crop the frame
        cropped_frame = frame[roi[1]:roi[1]+roi[3], roi[0]:roi[0] +roi[2]]

        cropped_resized_frame = Image.fromarray(cropped_frame.astype('uint8'))

        grayscale_image_cropped = cropped_resized_frame.convert('L')
 
        # Set the new size (half of original size in this example)
        new_size = newshape #defined above!!!
        # Resize the image using the Lanczos filter (you can use other filters too)
        resized_cropped_image = grayscale_image_cropped.resize(new_size, resample=Image.LANCZOS)
        
        # if(counter==100):
        #     resized_cropped_image.show()
        resize_crop_img = np.array(resized_cropped_image)
        cv2.equalizeHist( resize_crop_img,  resize_crop_img)
        # if(counter==100):
        #     pil_image = Image.fromarray(resize_crop_img)
        #     pil_image.show()

        if read_success == False:
            cv2.putText(frame, "Tracking failure occured", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
        else:
            # Load the image to be recognized
            # image = cv2.imread(frame, cv2.IMREAD_GRAYSCALE)
            
            # print ("Resizing Frame")
            # Resize the image to the same size as the training images
            # image = cv2.resize(frame, (720, 1280))
            
            # Convert the image to grayscale
#             gray_frame = cv2.cvtColor(resized_cropped_image, cv2.COLOR_BGR2GRAY)
            
            flattened_gray = np.ndarray.flatten(resize_crop_img) - mean            
            
            
            inferencedata = eigenmatrix @ flattened_gray
            
            # inferencedata = inferencedata/np.linalg.norm(inferencedata)
            
            # print ("Computing Eigenface confidence")
            # Compute the eigenface coefficients for the image
            # The predict function returns the predicted label and the confidence score
#             label, confidence = model.predict(cropped_frame)
            
            

        fps_value = int(cv2.getTickFrequency()/(cv2.getTickCount() - start))
        cv2.putText(frame, "FPS@"+str(fps_value), (500, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
#         cv2.putText(frame, "Predicted Label:"+str(label), (100, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
#         cv2.putText(frame, "Confidence Score:"+str(confidence), (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
        

        # for i in range()

        #1 Nearest Neighbor classifier
        distances = np.zeros(3*numPicsPerPerson)

        for i in range(numPicsPerPerson*3):
            distances[i] = np.linalg.norm(dataACR[i] - inferencedata)/100
            # distances[i] = np.abs(dataACR[i].dot(inferencedata))
            
        # label = labels[np.argmin(distances)]
        
        # distances = distances.reshape(3,numPicsPerPerson)
        # distances = distances.mean(axis= 1)
        
        label = labels[np.argmin(distances)]

        print(distances)
        # label = clf.predict(inferencedata.reshape(1,-1))
        
        # label = knn.predict(finaldata.reshape(1,-1))
        if (label == 0):
            cv2.putText(frame, "Predicted label: Aahan", (100, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
        elif (label == 1):
            cv2.putText(frame, "Predicted label: Chaz", (100, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
        elif (label == 2):
            cv2.putText(frame, "Predicted label: Rutvik", (100, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
        
#         cv2.putText(frame, "Confidence score: {}".format(confidence), (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)





    # Draw ROI Rectangle
    p1 = (int(roi[0]), int(roi[1]))
    p2 = (int(roi[0] + roi[2]), int(roi[1] + roi[3]))
    cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
    # Display result
    cv2.imshow("ECE420 Lab7", frame)
    

# # Print the predicted label and confidence score
# print('Predicted label: {}'.format(label))
# print('Confidence score: {}'.format(confidence))
# print("0 = Aahan; 1 = Chaz; 2 = Rutvik")

Frame Shape:  (480, 640, 3)
[5.83760613 4.928582   6.78740575 4.74802128 5.24644916 5.53180098
 6.06422756 7.71670555 6.55563807]
[5.77896533 5.00264738 6.82221228 4.75679707 5.27464287 5.54527353
 6.05885033 7.67145309 6.52334156]
[5.77896533 5.00264738 6.82221228 4.75679707 5.27464287 5.54527353
 6.05885033 7.67145309 6.52334156]
[5.77896533 5.00264738 6.82221228 4.75679707 5.27464287 5.54527353
 6.05885033 7.67145309 6.52334156]
[5.77896533 5.00264738 6.82221228 4.75679707 5.27464287 5.54527353
 6.05885033 7.67145309 6.52334156]
[5.92568049 5.0598611  6.95098866 4.91611663 5.4646249  5.71600941
 6.05048446 7.67932262 6.55649459]
[5.92568049 5.0598611  6.95098866 4.91611663 5.4646249  5.71600941
 6.05048446 7.67932262 6.55649459]
[6.08652675 5.13539477 7.1132856  5.09866165 5.71797204 5.96024886
 6.01234474 7.66618266 6.54862865]
[6.08652675 5.13539477 7.1132856  5.09866165 5.71797204 5.96024886
 6.01234474 7.66618266 6.54862865]
[6.08652675 5.13539477 7.1132856  5.09866165 5.7179720